In [1]:
import random as rd

from enum import Enum, auto
from dataclasses import dataclass

In [2]:
class Move(Enum):
    ROCK = auto()
    PAPER = auto()
    SCISSORS = auto()

class Outcome(Enum):
    PLAYER = auto()
    COMPUTER = auto()
    DRAW = auto()

In [3]:
class NaiveStrategy:
    def __init__(self, action_space=tuple(Move)):
        self.action_space = action_space

    def reset(self):
        pass
    
    def select_move(self):
        return rd.choice(self.action_space)

In [4]:
strategy = NaiveStrategy()

In [5]:
strategy.select_move()

<Move.SCISSORS: 3>

In [6]:
MOVE_BEATS = {
    Move.ROCK: Move.SCISSORS,
    Move.PAPER: Move.ROCK,
    Move.SCISSORS: Move.PAPER
}

In [7]:
def evaluate_round(player: Move, computer: Move):
    if player == computer:
        return Outcome.DRAW
    
    return Outcome.PLAYER if MOVE_BEATS[player] == computer else Outcome.COMPUTER

In [8]:
@dataclass(frozen=True, slots=True)
class RoundRecord:
    round_number: int
    player_move: Move
    computer_move: Move
    outcome: Outcome

In [9]:
class ResearchBasedStrategy:
    def __init__(
            self,
            action_space=tuple(Move),
            action_weights=[0.40, 0.35, 0.25]
    ):
        self.action_space = action_space
        self.action_weights = action_weights

    def select_move(self, match_history=None):
        # level 0
        if not match_history:
            return self.first_move()
        
        round_number = match_history[-1].round_number

        # level 1
        previous_round =  match_history[round_number]

        if previous_round.outcome == Outcome.PLAYER:
            return MOVE_BEATS[previous_round.computer_move]
        elif previous_round.outcome == Outcome.COMPUTER:
            return previous_round.player_move
        else:
            return self.first_move()
        
        # level 2
        #
        #
        #

    def first_move(self):
        return rd.choices(self.action_space, self.action_weights, k=1)[0]

In [10]:
strategy = ResearchBasedStrategy()

In [11]:
# rock (1) - paper (2) - scissors (3)

example_history = [
    # zapis przy pomocy wartości Enuma 
    RoundRecord(0, Move(1),       Move(2),       evaluate_round(Move(1),       Move(2))),
    RoundRecord(1, Move(2),       Move(2),       evaluate_round(Move(2),       Move(2))),
    # zapis przy pomocy nazw członków Enuma
    RoundRecord(2, Move.SCISSORS, Move.PAPER,    evaluate_round(Move.SCISSORS, Move.PAPER)),
    RoundRecord(3, Move.PAPER,    Move.SCISSORS, evaluate_round(Move.PAPER,    Move.SCISSORS)),
]

In [15]:
strategy.select_move()

<Move.ROCK: 1>

In [16]:
strategy.select_move(example_history)

<Move.PAPER: 2>

In [19]:
player_strategy = NaiveStrategy()
computer_strategy = ResearchBasedStrategy()
match_history = []

for round_idx in range(5):
    print(f"Runda numer: {round_idx}")

    player_move = player_strategy.select_move()
    computer_move = computer_strategy.select_move()

    print(f"Gracz rzucił: {player_move.name}")
    print(f"Komputer rzucił: {computer_move.name}")

    outcome = evaluate_round(player_move, computer_move)
    print(f"Wygrał {outcome.name}\n")

    match_history.append(
        RoundRecord(
            round_number=round_idx,
            player_move=player_move,
            computer_move=computer_move,
            outcome=outcome
        )
    )

Runda numer: 0
Gracz rzucił: ROCK
Komputer rzucił: PAPER
Wygrał COMPUTER

Runda numer: 1
Gracz rzucił: SCISSORS
Komputer rzucił: PAPER
Wygrał PLAYER

Runda numer: 2
Gracz rzucił: SCISSORS
Komputer rzucił: ROCK
Wygrał COMPUTER

Runda numer: 3
Gracz rzucił: PAPER
Komputer rzucił: SCISSORS
Wygrał COMPUTER

Runda numer: 4
Gracz rzucił: SCISSORS
Komputer rzucił: ROCK
Wygrał COMPUTER

